In [1]:
import os
import random
import warnings
from prophet.plot import add_changepoints_to_plot
import numpy as np
import pandas as pd
from prophet import Prophet

warnings.filterwarnings(action='ignore')

Importing plotly failed. Interactive plots will not work.


In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


seed_everything(42)  # Seed 고정

In [3]:
info = pd.read_csv('./building_info.csv')
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [4]:
train_df

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


## 가장 결과가 좋았던 외생변수 4종을 포함한 코드에 prameter 최적화를 수행한다

In [ ]:
test_submission = pd.DataFrame(columns=['num_date_time', 'answer'])
fillzero_train = train_df.fillna(0)
holidays = pd.DataFrame(columns=['holiday', 'ds'])
for i in range(1, 101):
    # 한건물당 168개씩 예측한다
    sample_train = fillzero_train.loc[fillzero_train["건물번호"] == i]
    target = sample_train["num_date_time"].str.split("_").apply(lambda x: x[1])
    sample_train["num_date_time"] = pd.to_datetime(target, format="%Y%m%d %H")
    data = sample_train.rename(columns={'num_date_time': 'ds', "전력소비량(kWh)": 'y'})[
        ['ds', 'y', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)']].reset_index(drop=True)
    # data['holidays'] = 0
    m = Prophet(
        # growth: Any = 'linear',
        #      changepoints: Any = None,
        #      n_changepoints: Any = 25,
             changepoint_range = 1,
        #      yearly_seasonality: Any = 'auto',
        #      weekly_seasonality: Any = 'auto',
        #      daily_seasonality: Any = 'auto',
             holidays = holidays,
        #      seasonality_mode: Any = 'additive',
        #      seasonality_prior_scale: Any = 10.0,
        #      holidays_prior_scale: Any = 10.0,
        #      changepoint_prior_scale = 0.01,
        #      mcmc_samples: Any = 0,
        #      interval_width: Any = 0.80,
        #      uncertainty_samples: Any = 1000,
        #      stan_backend: Any = None
    )
    m.add_country_holidays(country_name='KR')
    m.add_regressor('기온(C)')
    m.add_regressor('강수량(mm)')
    m.add_regressor('풍속(m/s)')
    m.add_regressor('습도(%)')
    m.fit(data)

    future = m.make_future_dataframe(periods=168, freq='1H')
    future['기온(C)'] = data['기온(C)']
    future['강수량(mm)'] = data['강수량(mm)']
    future['풍속(m/s)'] = data['풍속(m/s)']
    future['습도(%)'] = data['습도(%)']

    future.loc[future.shape[0] - 168:, '기온(C)'] = test_df.loc[(i - 1) * 168:i * 168 - 1, '기온(C)'].values
    future.loc[future.shape[0] - 168:, '강수량(mm)'] = test_df.loc[(i - 1) * 168:i * 168 - 1, '강수량(mm)'].values
    future.loc[future.shape[0] - 168:, '풍속(m/s)'] = test_df.loc[(i - 1) * 168:i * 168 - 1, '풍속(m/s)'].values
    future.loc[future.shape[0] - 168:, '습도(%)'] = test_df.loc[(i - 1) * 168:i * 168 - 1, '습도(%)'].values

    forecast = m.predict(future)

    answer = forecast[-168:].loc[:, ['ds', 'yhat']].rename(
        columns={"ds": "num_date_time", "yhat": "answer"}).reset_index(drop=True)

    answer["year"] = answer["num_date_time"].dt.year.astype(str)
    answer["month"] = answer["num_date_time"].dt.month.astype(str).str.zfill(2)
    answer["day"] = answer["num_date_time"].dt.day.astype(str).str.zfill(2)
    answer["hour"] = answer["num_date_time"].dt.hour.astype(str).str.zfill(2)
    answer["num_date_time"] = f"{i}_" + answer["year"] + answer["month"] + answer["day"] + ' ' + answer["hour"]
    test_submission = test_submission.append(answer)
test_submission = test_submission[["num_date_time", "answer"]]
test_submission.set_index("num_date_time", inplace=True)

13:13:33 - cmdstanpy - INFO - Chain [1] start processing
13:13:33 - cmdstanpy - INFO - Chain [1] done processing
13:13:34 - cmdstanpy - INFO - Chain [1] start processing
13:13:34 - cmdstanpy - INFO - Chain [1] done processing
13:13:34 - cmdstanpy - INFO - Chain [1] start processing
13:13:34 - cmdstanpy - INFO - Chain [1] done processing
13:13:35 - cmdstanpy - INFO - Chain [1] start processing
13:13:35 - cmdstanpy - INFO - Chain [1] done processing
13:13:36 - cmdstanpy - INFO - Chain [1] start processing
13:13:36 - cmdstanpy - INFO - Chain [1] done processing
13:13:36 - cmdstanpy - INFO - Chain [1] start processing
13:13:36 - cmdstanpy - INFO - Chain [1] done processing
13:13:37 - cmdstanpy - INFO - Chain [1] start processing
13:13:37 - cmdstanpy - INFO - Chain [1] done processing
13:13:38 - cmdstanpy - INFO - Chain [1] start processing
13:13:38 - cmdstanpy - INFO - Chain [1] done processing
13:13:38 - cmdstanpy - INFO - Chain [1] start processing
13:13:38 - cmdstanpy - INFO - Chain [1]

In [ ]:
m.train_holiday_names

In [ ]:
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

In [ ]:
fig = m.plot_components(forecast)

In [20]:
test_submission.to_csv("results/2023-08-15_3.csv")

## 제출결과
1. cap, floor 설정 및 logistic 분석 실시 : 11.6409051412(2023-08-13.csv)
2. 지난 최고점인 prophet linear모델에 holidays 추가 : 11.3840875952(2023-08-14_h.csv) 점수 상승
3. changepoint_range(과적합 방지용) = 0.95 : 11.3838344125(2023-08-14.csv) 점수 상승
4. changepoint_range(과적합 방지용) = 1 : 11.3809335269(2023-08-15.csv) 점수 상승
5. changepoint_prior_scale = 0.1 : 11.6773858778(2023-08-15_2.csv)
6. changepoint_prior_scale = 0.01 : 11.4329635772(2023-08-15_3.csv)